<a href="https://colab.research.google.com/github/maitysuvo19/internship_root2ai_classify_text/blob/main/Root2ai_classification_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/root2ai - Data.csv')

In [ ]:
df.head()

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain


In [ ]:
df=df.dropna()
df.reset_index(drop=True)

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain
...,...,...
22696,fourth study discusses blockchain technology e...,Reg Tech
22697,book finishes stating biggest issue emerging F...,Reg Tech
22698,people culture cess,Reg Tech
22699,author challenges execu tive lead change stop ...,Reg Tech


In [ ]:
df.isnull().any()

Text      False
Target    False
dtype: bool

In [ ]:
df["Target"].value_counts()/df.shape[0]

FinTech             0.376679
Cyber Security      0.116294
Bigdata             0.099863
Reg Tech            0.097176
credit reporting    0.077001
Blockchain          0.060570
Neobanks            0.047090
Microservices       0.042906
Stock Trading       0.034668
Robo Advising       0.032466
Data Security       0.015286
Name: Target, dtype: float64

In [ ]:
df['Target'] =df.Target.map({'FinTech':0, 'Cyber Security':1,'Bigdata':2,'Reg Tech':3,'credit reporting':4,'Blockchain':5,'Neobanks':6,'Microservices':7,'Stock Trading':8,'Robo Advising':9,'Data Security':10})
df = df[["Text","Target"]]
df.head()

,Text,Target
0,reserve bank forming expert committee based in...,5
1,director could play role financial system,5
2,preliminary discuss secure transaction study r...,5
3,security indeed prove essential transforming f...,5
4,bank settlement normally take three days based...,5


In [ ]:
df.head(5)

,Text,Target
0,reserve bank forming expert committee based in...,5
1,director could play role financial system,5
2,preliminary discuss secure transaction study r...,5
3,security indeed prove essential transforming f...,5
4,bank settlement normally take three days based...,5


In [ ]:
import sklearn
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
#splitting the data set
train, test = train_test_split(df, test_size=0.25)
print("Train DF: ",train.shape)
print("Test DF: ",test.shape)

Train DF:  (17025, 2)
Test DF:  (5676, 2)


In [ ]:
#Make the necessary imports
import sys
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential
from keras.initializers import Constant

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

In [ ]:
GLOVE_DIR='/content/drive/MyDrive/nlp data/glove.6B.100d.txt'

In [ ]:
labels_index = {'FinTech':0, 'Cyber Security':1,'Bigdata':2,'Reg Tech':3,'credit reporting':4,'Blockchain':5,'Neobanks':6,'Microservices':7,'Stock Trading':8,'Robo Advising':9,'Data Security':10}

In [ ]:
train_texts=train['Text'].tolist()
train_labels=train['Target'].tolist()
test_texts=test['Text'].tolist()
test_labels=test['Target'].tolist()

In [ ]:
#Just to see how the data looks like. 
print(train_texts[0])
print(train_labels[0])
print(test_texts[0])
print(test_labels[0])

initial capital start phase bank could pose greater risk financial progressively reduce amount funds available
0
promise data
0


In [ ]:
#Vectorize these text samples into a 2D integer tensor using Keras Tokenizer
#Tokenizer is fit on training data only, and that is used to tokenize both train and test data.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10271 unique tokens.


In [ ]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
 #initial padding of 0s, until vector is of size MAX_SEQUENCE_LENGTH
trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels = to_categorical(np.asarray(train_labels))
test_labels = to_categorical(np.asarray(test_labels))

# split the training data into a training set and a validation set
indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])
x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]
#This is the data we will use for CNN and RNN training
print('Splitting the train data into train and valid is done')

Splitting the train data into train and valid is done


In [ ]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open(GLOVE_DIR) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
#print(embeddings_index["google"])

# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.
Preparing of embedding matrix is done


# 1D CNN Model with pre-trained embedding

In [ ]:
print('Define a 1D CNN model.')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(labels_index), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=5, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Define a 1D CNN model.
Epoch 1/5
107/107 [==============================] - 38s 44ms/step - loss: 2.0121 - acc: 0.3809 - val_loss: 1.7603 - val_acc: 0.4564
Epoch 2/5
107/107 [==============================] - 4s 37ms/step - loss: 1.6329 - acc: 0.4950 - val_loss: 1.5965 - val_acc: 0.5037
Epoch 3/5
107/107 [==============================] - 4s 37ms/step - loss: 1.4497 - acc: 0.5463 - val_loss: 1.5791 - val_acc: 0.5160
Epoch 4/5
107/107 [==============================] - 4s 37ms/step - loss: 1.3103 - acc: 0.5810 - val_loss: 1.5697 - val_acc: 0.5151
Epoch 5/5
178/178 [==============================] - 1s 7ms/step - loss: 1.4978 - acc: 0.5400
Test accuracy with CNN: 0.5399929285049438


# 1D CNN model with training your own embedding

In [ ]:
print("Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings")
cnnmodel = Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(labels_index), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=5, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings
Epoch 1/5
107/107 [==============================] - 8s 70ms/step - loss: 2.0768 - acc: 0.3586 - val_loss: 1.8881 - val_acc: 0.3997
Epoch 2/5
107/107 [==============================] - 7s 67ms/step - loss: 1.7713 - acc: 0.4270 - val_loss: 1.6335 - val_acc: 0.5019
Epoch 3/5
107/107 [==============================] - 7s 67ms/step - loss: 1.4385 - acc: 0.5538 - val_loss: 1.5251 - val_acc: 0.5278
Epoch 4/5
107/107 [==============================] - 7s 67ms/step - loss: 1.2467 - acc: 0.5987 - val_loss: 1.6369 - val_acc: 0.4881
Epoch 5/5
178/178 [==============================] - 1s 7ms/step - loss: 1.6259 - acc: 0.5217
Test accuracy with CNN: 0.5216701626777649


# LSTM Model with training your own embedding

In [ ]:
print("Defining and training an LSTM model, training embedding layer on the fly")

#model
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(11, activation='softmax'))
rnnmodel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, training embedding layer on the fly
Training the RNN
Epoch 1/5
426/426 [==============================] - 1380s 3s/step - loss: 1.9260 - accuracy: 0.4056 - val_loss: 1.2508 - val_accuracy: 0.6132
Epoch 2/5
426/426 [==============================] - 1381s 3s/step - loss: 0.9668 - accuracy: 0.6969 - val_loss: 1.0930 - val_accuracy: 0.6655
Epoch 3/5
426/426 [==============================] - 1378s 3s/step - loss: 0.6241 - accuracy: 0.8029 - val_loss: 1.1311 - val_accuracy: 0.6529
Epoch 4/5
426/426 [==============================] - 1371s 3s/step - loss: 0.4284 - accuracy: 0.8627 - val_loss: 1.2478 - val_accuracy: 0.6455
Epoch 5/5
178/178 [==============================] - 38s 213ms/step - loss: 1.3160 - accuracy: 0.6506
Test accuracy with RNN: 0.650634229183197


LSTM Model using pre-trained Embedding Layer

In [ ]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(11, activation='softmax'))
rnnmodel2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel2.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=256)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, using pre-trained embedding layer
Training the RNN
Epoch 1/5
54/54 [==============================] - 180s 3s/step - loss: 2.0296 - accuracy: 0.3652 - val_loss: 1.6569 - val_accuracy: 0.4849
Epoch 2/5
54/54 [==============================] - 175s 3s/step - loss: 1.6122 - accuracy: 0.4954 - val_loss: 1.4690 - val_accuracy: 0.5272
Epoch 3/5
54/54 [==============================] - 176s 3s/step - loss: 1.4779 - accuracy: 0.5249 - val_loss: 1.4043 - val_accuracy: 0.5539
Epoch 4/5
54/54 [==============================] - 176s 3s/step - loss: 1.4038 - accuracy: 0.5509 - val_loss: 1.3733 - val_accuracy: 0.5583
Epoch 5/5
23/23 [==============================] - 6s 255ms/step - loss: 1.3580 - accuracy: 0.5514
Test accuracy with RNN: 0.5514446496963501
